<h1 align="center">
  <a href="https://uptrain.ai">
    <img width="300" src="https://user-images.githubusercontent.com/108270398/214240695-4f958b76-c993-4ddd-8de6-8668f4d0da84.png" alt="uptrain">
  </a>
</h1>

<h1 style="text-align: center;">Monitoring a Trip Time prediction model</h1>

**Overview**: In this example, we consider a regression task where we want to predict the trip time given the trip distance, number of passengers, booking date, vendor ID, etc. 

**Dataset**: We use the [New York City Taxi Trip Duration dataset](https://www.kaggle.com/competitions/nyc-taxi-trip-duration/data) from Kaggle, where the data contains features such as vendor_id, pickup_datetime, passenger_count, pickup_location, drop_location, etc., and the trip durations (in seconds). We want to train and ML model that takes the input features and predicts the trip duration. 

**Monitoring**: In this notebook, we will see how we can use UpTrain package to monitor model accuracy, run data integrity checks, and add SHAP in UpTrain dashboard to explain model predictions.

#### Install required packages for this example [LightGBM]
Note: For mac users, try conda installation if pip doesn't work

In [1]:
# !pip install lightgbm

#### Let's first import all the required packages

In [2]:
import pandas as pd
from sklearn import model_selection, metrics
from lightgbm import LGBMRegressor
import time
import uptrain
import os
from helper_funcs import download_dataset, process, pretty

import matplotlib
matplotlib.use('TkAgg')

### Download and prepare the Taxi Trip Duration Dataset

In [3]:
data_file = 'trip_duration_dataset.csv'
download_dataset(data_file)
df_train = pd.read_csv(data_file)

# Divide the data into input data to the model and targets 
y = df_train["trip_duration"]
X = df_train.drop(["trip_duration"], axis = 1)

Data file exists. Skipping download.


In [4]:
# Divide data into train-test splits
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1)

# Get ids for data to locate them with their ids in the UpTrain dashboard
test_ids = list(X_test['id'])

#### Let's see how the input looks like

In [5]:
# Cleanup data to, for instance, find distance from latitude and longitude
X_train = process(X_train)
X_test = process(X_test)

# Let's see how the data looks
X_train.head(3)

,vendor_id,passenger_count,store_and_fwd_flag,dist,pickup_year,pickup_month,pickup_day,pickup_dow,pickup_hour,pickup_min,pickup_sec
1397389,1,2,0,0.013406,2016,2,3,2,18,59,1
966155,2,1,0,0.060024,2016,4,23,5,0,53,28
256642,2,1,0,0.005566,2016,4,1,4,16,3,22


### Train our LightGBM Regression model

In [6]:
model = LGBMRegressor(n_estimators=500) 
model.fit(X_train, y_train)

LGBMRegressor(n_estimators=500)

#### Check the training MAPE (Mean Absolute Percentage Error) of our model

In [7]:
preds = abs(model.predict(X_train))
mae_train = metrics.mean_absolute_error(y_train, preds)
print(f"Training Mean Percentage Error: {mae_train:.2f} seconds")
mape_train = metrics.mean_absolute_percentage_error(y_train, preds)
print(f"Training Mean Absolute Percentage Error: {mape_train*100:.2f} %")

Training Mean Percentage Error: 415.74 seconds
Training Mean Absolute Percentage Error: 75.34 %


### Defining Monitors over test data using the UpTrain Framework

Next, we define monitors over our ML model in test/production with UpTrain.

#### Accuracy monitors
We define Mean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE) accuracy monitors for our regression task

In [8]:
# To monitor MAE
mae_monitor = {       
    "type": uptrain.Monitor.ACCURACY,
    "measurable_args": {
        'type': uptrain.MeasurableType.MAE,
    },
}

# To monitor  (MAPE)
mape_monitor = {
    "type": uptrain.Monitor.ACCURACY,
    "measurable_args": {
        'type': uptrain.MeasurableType.MAPE,
    },
}

#### SHAP explanability
SHAP (SHapley Additive exPlanations) is a game theoretic approach to explain the ML model predictions and it available as a [python package](https://github.com/slundberg/shap). Through UpTrain dashboard, we will use SHAP to explain our model's preditions.

In [9]:
# To visualize the SHAP explanability in dashboard
shap_visual = {
    "type": uptrain.Visual.SHAP,
    "model": model,
    # Limit the number of points for which SHAP values are calculated to reduce runtime.
    "shap_num_points": 10000,
    "id_col": "id"
}

#### Data integrity monitor
We can also define data integrity checks over our test data. One obvious check is that the number of passengers should >= 1.

In [10]:
# Data integrity monitor to check if feature 'pasenger_count' is >=1
data_integrity_monitor =  {
    "type": uptrain.Monitor.DATA_INTEGRITY,
    "measurable_args": {
        'type': uptrain.MeasurableType.INPUT_FEATURE,
        'feature_name': 'passenger_count'
    },
    "integrity_type": "greater_than",
    "threshold": 1
}

### Define the checks in config and pass it to the UpTrain framework

In [11]:
cfg = {
    "checks": [mae_monitor, mape_monitor, data_integrity_monitor, shap_visual],
    "logging_args": {"st_logging": True},
}
framework = uptrain.Framework(cfg_dict=cfg)
pretty(cfg)

Deleting the folder:  uptrain_smart_data
Deleting the folder:  uptrain_logs
- checks:
	- type:
		Monitor.ACCURACY
	- measurable_args:
		- type:
			MeasurableType.MAE
	- type:
		Monitor.ACCURACY
	- measurable_args:
		- type:
			MeasurableType.MAPE
	- type:
		Monitor.DATA_INTEGRITY
	- measurable_args:
		- type:
			MeasurableType.INPUT_FEATURE
		- feature_name:
			passenger_count
	- integrity_type:
		greater_than
	- threshold:
		1
	- type:
		Visual.SHAP
	- model:
		LGBMRegressor(n_estimators=500)
	- shap_num_points:
		10000
	- id_col:
		id
- logging_args:
	- st_logging:
		True


### Launch the model in Production

In [ ]:
batch_size = 2000
cols = list(X_test.columns)
for idx in range(int(len(X_test)/batch_size)):
    
    indices = range(idx*batch_size, (idx+1)*batch_size)
    this_elems = X_test.iloc[idx*batch_size: (idx+1)*batch_size, :]
    this_preds = abs(model.predict(this_elems))
    
    # Add ids to corresponding data points to preserve them in dashboard
    this_elems = this_elems.assign(id=list(test_ids[idx*batch_size: (idx+1)*batch_size]))
    
    # Log input and outputs to the UpTrain framework
    ids = framework.log(inputs=this_elems, outputs=this_preds)
    
    # Attach ground truth
    ground_truth = list(y_test[idx*batch_size: (idx+1)*batch_size])
    framework.log(identifiers=ids, gts=ground_truth)
    
    time.sleep(0.01)

### SHAP Explanability at the UpTrain Dashboard

As we can notice from the dashboard, we get two plots for SHAP explanability. The plot on the left is the average weight of each feature. As expected, the feature "dist" (that represents the distance of the trip) has the biggest impact on the trip duration prediction. Moreover, pickup hour and taxi vendor id also somewhat affect the trip duration. 

On the right, we can see how the model arrive at the prediction for any data-point (in this case ID `id1787379`). Due to the low distance of the trip, the feature "dist" contributed `-256.06` to the overall trip duration. 

<img src="https://uptrain-demo.s3.us-west-1.amazonaws.com/ride_estimation/shap_featurewise.png" width="510"/> <img src="https://uptrain-demo.s3.us-west-1.amazonaws.com/ride_estimation/shap_datapoint.png" width="450"/> 

### Demo Video: SHAP explainability
We have added a small video on how it looks below.

In [13]:
from IPython.display import Video
video = Video("https://uptrain-demo.s3.us-west-1.amazonaws.com/ride_estimation/4_Explanability+recording.mov")
video.width = 600
video

### MAE and MAPE Accuracy

<img src="https://uptrain-demo.s3.us-west-1.amazonaws.com/ride_estimation/mae.png" width="450"/> <img src="https://uptrain-demo.s3.us-west-1.amazonaws.com/ride_estimation/mape.png" width="457"/> 

### Data Integrity
We added a data integrity check for feature passenger_count >=1. We observe that data integrity is maintained through the test dataset (as data integrity is close to one)


<img src="https://uptrain-demo.s3.us-west-1.amazonaws.com/ride_estimation/data_integrity_passenger_count.png" width="650"/>